<a href="https://colab.research.google.com/github/ntshrocha/MAB605-2019.1/blob/master/Recupera%C3%A7%C3%A3o_na_Web_Natasha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercício 9 - Implementação do PageRank

In [0]:
def get_pagerank(graph, beta=0.8, threshold=10e-5):
  T = len(graph)
  PR = dict.fromkeys(graph, 1/T) # initialize PR with weight 1/T
  oldPR = dict.copy(PR)
  
  converged = dict.fromkeys(graph, False) # tells if PR for a given page has converged
  
  while(any((c == False) for c in converged.values())):
    for page in PR:
      oldPR[page] = PR[page]
      PR[page] = (1 - beta)/T
      
      for node in graph:
        if page in graph[node]:
          PR[page] += beta*PR[node]/len(graph[node])

      converged[page] = abs(oldPR[page] - PR[page]) <= threshold
    
  return {k: '%.3f'%(v) for k, v in PR.items()} # truncate to 3 decimal places


# EXEMPLO ######################################################################

G = {"A": ["B", "C"],
     "B": ["C"],
     "C": ["A"],
     "D": ["C"],
    }

get_pagerank(G)

{'A': '0.363', 'B': '0.195', 'C': '0.392', 'D': '0.050'}

## Exercício 10 - Implementação do HITS

In [0]:
# TRANSPOSTA DE UM GRAFO #######################################################
def transpose(graph):
  T = {key: [] for key in graph.keys()}
  
  for node in graph:
    for adj in graph[node]:
      T[adj].append(node)
      
  return T

# EXEMPLO ######################################################################
print(G)
transpose(G)

{'A': ['B', 'C'], 'B': ['C'], 'C': ['A'], 'D': ['C']}


{'A': ['C'], 'B': ['A'], 'C': ['A', 'B', 'D'], 'D': []}

In [0]:
# NORMALIZANDO UM VETOR ########################################################
def normalize(my_dict):
  key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
  key_max = max(my_dict.keys(), key=(lambda k: my_dict[k]))
  
  normalized = dict.fromkeys(my_dict)
  
  for key in my_dict:
    normalized[key] = (my_dict[key] - my_dict[key_min]) / (my_dict[key_max] - my_dict[key_min])
  
  return normalized

# EXEMPLO ######################################################################
normalize({'A': 1, 'B': 2, 'C': 4, 'D': 0})

{'A': 0.25, 'B': 0.5, 'C': 1.0, 'D': 0.0}

In [0]:
# ALGORITMO HITS ###############################################################
def get_hits(graph, k):
  hub = dict.fromkeys(graph, 1)
  auth = dict.fromkeys(graph, 1)
  
  graph_T = transpose(G)
  
  for i in range(0, k):
    # Atualiza hubs
    for node in graph:
      hub[node] = 0
      for adj in graph[node]:
        hub[node] += auth[adj]
    hub = normalize(hub)
    
    # Atualiza autoridades
    for node in graph_T:
      auth[node] = 0
      for adj in graph_T[node]:
        auth[node] += hub[adj]
    auth = normalize(auth)
    
  return {"hub": hub, "auth": auth}


# EXEMPLO ######################################################################

G = {"A": ["B", "C"],
     "B": ["C"],
     "C": ["A"],
     "D": ["C"],
    }

get_hits(G, 650)

{'auth': {'A': 0.0, 'B': 0.4142135623730951, 'C': 1.0, 'D': 0.0},
 'hub': {'A': 1.0, 'B': 0.7071067811865475, 'C': 0.0, 'D': 0.7071067811865475}}